In [113]:
import pandas as pd
import numpy as np

In [2]:
# thêm title cho file excel, vị trí mặc định tại 1,2
import openpyxl
def add_title(file_name, title, sheet="Sheet1"):
    wb = openpyxl.load_workbook(file_name)
    old_sheet = wb.get_sheet_by_name(sheet)
    old_sheet.cell(row=1, column=2).value = title
    wb.save(file_name)
# lấy title cho file
def get_title(file_name, sheet="Sheet1"):
    wb = openpyxl.load_workbook(file_name)
    old_sheet = wb.get_sheet_by_name(sheet)
    value = old_sheet.cell(1, 2).value
    return value

In [3]:
# hàm lưu dataframe
def save_dataframe(dataframe, file_name, sheet = 'Sheet1'):
    writer = pd.ExcelWriter(file_name)
    dataframe.to_excel(writer, sheet ,index=False, startrow=1) # bỏ trống hàng đầu tiên làm title
    writer.save()
def read_excel_file(file_name, sheet = 'Sheet1'):
    df = pd.read_excel(file_name,sheet,skiprows=[0]) # không đọc hàng đầu tiên (title)
    return df

In [4]:
# tạo file chỉ gồm các header
def create_empty_excel_file(columns, file_name, sheet = 'Sheet1'):
    df = pd.DataFrame(columns=columns)
    save_dataframe(df, file_name, sheet)
    return df

In [5]:
# thêm 1 hàng vào file
def insert_row(file_name, row, sheet = 'Sheet1', save = False):
    df = pd.read_excel(file_name)
    columns = df.columns.values.tolist()
    row=pd.Series(row,columns)
    df = df.append([row],ignore_index=True)
    if(save == True):
        save_dataframe(df, file_name, sheet)
    return df

In [6]:
def insert_row_to_df(df, row):
    columns = df.columns.values.tolist()
    row=pd.Series(row,columns)
    df = df.append([row],ignore_index=True)
    return df

In [7]:
# thêm dữ liệu từ file1 vào file2, kết quả lưu tại file2
def insert_data_from_file(from_file, to_file):
    df = read_excel_file(to_file)
    df1 = read_excel_file(from_file)
    df = df.append(df1)
    save_dataframe(df,to_file)
    return df

In [8]:
# tạo file chỉ có header:
columns = ['col1','col2','col3']
create_empty_excel_file(columns, "empty_file.xlsx")
emptydf = pd.read_excel("empty_file.xlsx")
emptydf

,col1,col2,col3


In [9]:
# thêm 1 hàng vào file
row = ['a','b','c']
insert_row("empty_file.xlsx", row)
emptydf = pd.read_excel("empty_file.xlsx")
emptydf

,col1,col2,col3


In [10]:
def get_fruits_table():
    return read_excel_file('2018/HangHoa.xlsx',sheet = 'MAU')
fruit_table = get_fruits_table()

In [11]:
import datetime

def get_date_current():
    now = datetime.datetime.now()
    return str(now.day)+"-"+str(now.month)+"-"+str(now.year)
get_date_current()

'7-10-2018'

In [12]:
def add_to_nhapCT(ngay,tensp,dvt,sl,dongia):
    global dataCT
    date_format = '%Y-%m-%d'
    if check_fruit_exist(tensp):
        ngay = datetime.datetime.strptime(ngay, date_format).date()
        thanhtien = int(dongia)*int(sl)
        row = [ngay,tensp,dvt,sl,dongia,thanhtien]
        dataCT = insert_row_to_df(dataCT, row)
    else:
        print("sản phẩm vừa nhập không có trong danh sách.")

In [67]:
def get_DVT(fruit_name):
    df = get_fruits_table()
    series1 = df[(df["TÊN SP"] == fruit_name)]
    return series1.iloc[0][1]
get_DVT("TÁO GALA MỸ")
def check_fruit_exist(fruit_name):
    global fruit_table
    return fruit_name in fruit_table["TÊN SP"].values


True

In [124]:
#dữ liệu của 1 loại quả trong 1 sheet của NhapTong.xlsx
from numpy import nan as Nan
def data_of_fruit_month(dataCT,fruit_name, month, year):
    null_row = [Nan]*34
    for i in range(1,32):
        str_to_cmp = "-"+str(month).zfill(2)+"-"+str(i).zfill(2)
        df1 = dataCT[(dataCT["TÊN SP"] == fruit_name) & (dataCT["NGÀY"].str.contains(str_to_cmp))]
        null_row[i+1] = df1["SL"].sum() 
    sm = 0
    for i in range(len(null_row)):
        try:
            sm = sm+int(null_row[i])
        except:
            continue
    null_row[33] = sm
    null_row[0] = fruit_name
    null_row[1] = get_DVT(fruit_name)
    return null_row


In [126]:
# tạo 1 sheet của file NhapTong.xlsx
def data_all_fruit_month(dataCT,month,year):
    columns = ['TÊN SP','ĐVT']   
    for i in range(1,32):
        columns.append(str(i))
    columns.append("TỔNG")
    df = pd.DataFrame(columns=columns)
    for fruit in get_fruits_table()["TÊN SP"]:
        row=pd.Series(data_of_fruit_month(dataCT,fruit, month, year),columns)
        df = df.append([row],ignore_index=True)
    return df

In [56]:
#tạo file Nhaptong.xlsx
def create_NhapTong(year):
    for i in range(1,13):
        try:
            dataCT = read_excel_file("nhapCT.xlsx", sheet = "T"+str(i))
            save_dataframe(data_all_fruit_month(dataCT,i,year), str(year)+"/Nhaptong.xlsx", sheet = "T"+str(i))
        except:
            print("không tìm thấy dữ liệu tháng %d", i)

In [17]:
work_folder = "2018"

In [127]:
data_all_fruit_month(dataCT,1,2018)

,TÊN SP,ĐVT,1,2,3,4,5,6,7,8,...,23,24,25,26,27,28,29,30,31,TỔNG
0,TÁO ENVY SIZE 24,THÙNG,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,TÁO ENVY SIZE 40,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TÁO ENVY SIZE 45,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TÁO JULIET SIZE 72,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TÁO JULIET SIZE 80,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,TÁO JULIET SIZE 88,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,TÁO JULIET SIZE 100,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,TÁO JULIET SIZE 113,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,TÁO GALA MỸ,THÙNG,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
9,TÁO AMBROSIA,THÙNG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def read_dataCT(work_folder):
    sheet = "T1"
    file = work_folder+"/nhapCT.xlsx"
    df = pd.read_excel(file,sheet,skiprows=[0],usecols=range(6)) # không đọc hàng đầu tiên (title)
    return df

In [75]:
dataCT = read_dataCT(work_folder)
dataCT["NGÀY"] = dataCT["NGÀY"].astype(str)

In [54]:
dataCT

,NGÀY,TÊN SP,ĐVT,SL,ĐON GIÁ,THÀNH TiỀN
0,2018-01-01,TÁO ENVY SIZE 24,THÙNG,3,100,300
1,2018-01-01,TÁO GALA MỸ,THÙNG,5,200,1000
2,2018-01-03,TÁO Ống 265,Ống,3,150,450
3,2018-01-04,TÁO Ống 265,Ống,2,150,300
4,2018-01-10,TÁO Ống 265,Ống,3,150,450
5,2018-01-22,TÁO Ống 265,Ống,1,150,150
6,2018-01-03,TÁO ENVY SIZE 24,THÙNG,3,100,300


In [21]:
add_to_nhapCT("2018-1-3","TÁO ENVY SIZE 24","THÙNG",3,100)

In [23]:
dataCT

,NGÀY,TÊN SP,ĐVT,SL,ĐON GIÁ,THÀNH TiỀN
0,2018-01-01,TÁO ENVY SIZE 24,THÙNG,3,100,300
1,2018-01-01,TÁO GALA MỸ,THÙNG,5,200,1000
2,2018-01-03,TÁO Ống 265,Ống,3,150,450
3,2018-01-04,TÁO Ống 265,Ống,2,150,300
4,2018-01-10,TÁO Ống 265,Ống,3,150,450
5,2018-01-22,TÁO Ống 265,Ống,1,150,150
6,2018-01-03,TÁO ENVY SIZE 24,THÙNG,3,100,300
